In [1]:
import qforte as qf
from qforte import *
import numpy as np

$$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\cre}[1]{\hat{a}^{\dagger}_{#1}}
\newcommand{\ann}[1]{\hat{a}_{#1}}
$$

# Implementing a Projective Quantum Eigensolver (PQE) with QForte

In this tutorial we will show how QForte can be used to implement quantum algorithms for electronic structure problems. We will use the H2 model system as an example, and will show how to implement the projective quantum eigensolver (PQE) to determine the FCI energy.

## PQE Theory

The projective quantum eigensolver is an alternative to the widely used variational quantum eigensolver (VQE).
Like in VQE, we approximate the ground state using a trial state $\ket{\tilde{\Psi}(\mathbf{t})}= \hat{U}(\mathbf{t}) \ket{\Phi_0}$.
After inserting the definition of the trial state in the Schr\"{o}dinger equation and left-multiplying by $\hat{U}^\dagger(\mathbf{t})$, we obtain the condition

\begin{equation}
\hat{U}^\dagger(\mathbf{t}) \hat{\mathcal{H}} \hat{U}(\mathbf{t}) \ket{\Phi_0} = E \ket{\Phi_0}.
\end{equation}

Projection onto the reference state $\Phi_0$ yields the PQE energy ($E_\text{PQE}$)

\begin{equation}
E_\text{PQE}(\mathbf{t}) = \bra{\Phi_0} \hat{U}^\dagger(\mathbf{t}) \hat{\mathcal{H}} \hat{U}(\mathbf{t}) \ket{\Phi_0},
\end{equation}

a quantity that is still an upper bound to the exact ground state energy.
Projections onto the complete set of orthonormal many-body basis functions complementary to $\Phi_0$, here denoted as $Q = \{\Phi_\mu \}$, yields a set of residual conditions

\begin{equation}
r_\mu(\mathbf{t}) \equiv \bra{\Phi_\mu} \hat{U}^\dagger(\mathbf{t}) \hat{\mathcal{H}} \hat{U}(\mathbf{t}) \ket{\Phi_0} = 0 \quad \forall \Phi_\mu \in Q,
\end{equation}

where $r_\mu$ is an element of the residual vector and $\mu$ runs over all elements of the many-body basis.
The above equations form a system of nonlinear equations in the parameter vector $\mathbf{t}$, that may be solved via a classical iterative solver.

### The disentangeld unitary coupled cluster ansatz

One of the most common state-preparation unitary circuits [$\hat{U}(\mathbf{t})$] in both VQE and PQE is the disentangled (or factorized) unitary coupled cluster (dUCC) ansatz.
In UCC, the reference state is an easily-prepared single determinant $\ket{\Phi_0} = \ket{\psi_1 \psi_2 \cdots}$ specified by the occupied spin orbitals $\{ \psi_i \}$.
 A UCC unitary is parameterized using a pool
of anti-Hermitian operators $\mathcal{P} = \{  \hat{\kappa}_\mu : \mu =1 ,\ldots, N_\mathrm{op}^\mathrm{pool} \}$.
A generic anti-Hermitian operator $\hat{\kappa}_\mu = \hat{\tau}_\mu - \hat{\tau}_\mu^\dagger$ is defined in terms of the particle-hole excitation operators
$ \hat{\tau}_\mu \equiv  \hat{\tau}_{ij\cdots}^{ab\cdots} = \cre{a} \cre{b} \cdots \ann{j} \ann{i}$.
Note that we have re-interpreted $\mu$ as the multi-index $\mu \equiv ((i,j,..),(a,b,..))$ of unique excitations from hole/occupied ($\psi_i \psi_j \cdots$) to particle/unoccupied ($\psi_a \psi_b \cdots$) spin orbitals.
Using this parameterization, when a cluster operator $ \hat{\kappa}_\mu$ acts on the reference, it generates elements of the many-body basis (excited determinants) of the form

\begin{equation}
\ket{\Phi_\mu} = \hat{\kappa}_\mu \ket{\Phi_0} = \ket{\Phi_{ij\cdots}^{ab\cdots}},
\end{equation}

and since in the case of a UCC (or dUCC) ansatz there is a 1-to-1 correspondence between operators and determinants, we may label them with the same index.
Note that this operator basis satisfies the orthonormality condition 

\begin{equation}
\bra{\Phi_0} \hat{\kappa}^\dagger_\mu \hat{\kappa}_\nu \ket{\Phi_0} = \langle \Phi_\mu \ket{\Phi_\nu} = \delta_{\mu\nu}.
\end{equation}

In traditional UCC, the wave function is generated by an exponential operator

\begin{equation}
\hat{U}(\mathbf{t}) = e^{\hat{\sigma}} = e^{\sum_\mu t_\mu \hat{\kappa}_\mu},
\end{equation}

assuming the cluster amplitudes $t_\mu$ are real.

In principle it is possible to construct a circuit that exactly implements the action of the UCC operator defined in, but in practice it is common to use a unitary with a simpler, and shallower, circuit.
This is frequently accomplished using a factorized (disentangled) form of the UCC ansatz

\begin{equation}
\hat{U}(\mathbf{t})=
 \prod_\mu e^{ t_\mu \hat{\kappa}_\mu}.
\end{equation}

Because the operators $\hat{\kappa}_\mu$ do not commute, an ansatz of the disentangled form is uniquely defined by an \textit{ordered} set (or subset) of operators $\mathcal{A} = ( \hat{\kappa}_{\mu_i}: i = 1, \ldots, N_\mathrm{op} )$ built from the operator pool $\mathcal{P}$.
The operators in $\mathcal{A}$ are then used form an ordered product of exponential unitaries

\begin{equation}
\label{eq:qucc}
\hat{U}(\mathbf{t})
= e^{t_{\mu_1} \hat{\kappa}_{\mu_1}}  e^{t_{\mu_2} \hat{\kappa}_{\mu_2}} \cdots e^{t_{\mu_{N_\mathrm{op}}} \hat{\kappa}_{\mu_{N_\mathrm{op}}}},
\end{equation}

where $t_{\mu_i}$ is the amplitude corresponding to the operator $\hat{\kappa}_{\mu_i}$.

### The dUCC-PQE update equation

To solve the PQE equations we measure the residuals corresponding to the operators contained in $\mathcal{A}$ on a quantum computer and update the parameter vector using a simple quasi-Newton iteration approach
\begin{equation}
\label{eq:fixed_point}
t_\mu^{(n +1)} = t_\mu^{(n)} + \frac{r^{(n)}_\mu}{\Delta_\mu},
\end{equation}
where the superscript "$(n)$" indicates the amplitude at iteration $n$.
The quantities $\Delta_\mu$ are standard Moller-Plesset denominators $\Delta_\mu \equiv \Delta_{ij\cdots}^{ab\cdots} = \epsilon_i + \epsilon_j + \ldots -\epsilon_a -\epsilon_b \ldots$ where $\epsilon_i$ are Hartree-Fock orbital energies.
This update equation is derived in using Newton's method and taking the leading contributions to the Jacobin to be the diagonal elements of the Fock operator.


## Implementation with QForte

To begin we will need to obtain the QForte molecule object. We will use the same simple hydrogen example from the previous totuorial.

> Get molecular Hamiltonian for H2.

In [2]:
# Define the reference and geometry lists.
geom = [('H', (0., 0., 0.0)), ('H', (0., 0., 0.75))]

# Get the molecule object that now contains both the fermionic and qubit Hamiltonians.
H2mol = system_factory(build_type='psi4', mol_geometry=geom, basis='sto-3g')

 ==> Psi4 geometry <==
-------------------------
0  1
H  0.0  0.0  0.0
H  0.0  0.0  0.75
symmetry c1
units angstrom


We will also need to construct a unitary circuit that sets the vacuum to the Hartree-Fock state for the H2 molecule.

> Build a HF state circuit ($\hat{U}_\mathrm{HF}$) for H2 and show that it constructs the correct state. 

In [3]:
# Initialize the circuit.
Uhf = qf.Circuit()
Uhf.add_gate(qf.gate('X', 0, 0))
Uhf.add_gate(qf.gate('X', 1, 1))

# Initialize a QuantumComputer
print('\nThe vaccume state:')
QC = qf.Computer(4)
print(QC)

# Set the QuantumComputer to the Hartree-Fock state using Uhf
print('\nThe Hartree-Fock state:')
QC.apply_circuit(Uhf)
print(QC)


The vaccume state:
Computer(
  (1.000000 +0.000000 i) |0000>
  (0.000000 +0.000000 i) |1000>
  (0.000000 +0.000000 i) |0100>
  (0.000000 +0.000000 i) |1100>
  (0.000000 +0.000000 i) |0010>
  (0.000000 +0.000000 i) |1010>
  (0.000000 +0.000000 i) |0110>
  (0.000000 +0.000000 i) |1110>
  (0.000000 +0.000000 i) |0001>
  (0.000000 +0.000000 i) |1001>
  (0.000000 +0.000000 i) |0101>
  (0.000000 +0.000000 i) |1101>
  (0.000000 +0.000000 i) |0011>
  (0.000000 +0.000000 i) |1011>
  (0.000000 +0.000000 i) |0111>
  (0.000000 +0.000000 i) |1111>
 )

The Hartree-Fock state:
Computer(
  (0.000000 +0.000000 i) |0000>
  (0.000000 +0.000000 i) |1000>
  (0.000000 +0.000000 i) |0100>
  (1.000000 +0.000000 i) |1100>
  (0.000000 +0.000000 i) |0010>
  (0.000000 +0.000000 i) |1010>
  (0.000000 +0.000000 i) |0110>
  (0.000000 +0.000000 i) |1110>
  (0.000000 +0.000000 i) |0001>
  (0.000000 +0.000000 i) |1001>
  (0.000000 +0.000000 i) |0101>
  (0.000000 +0.000000 i) |1101>
  (0.000000 +0.000000 i) |0011>
  (0

For H2 there is only one double excitation operator $\hat{\kappa} = \cre{2} \cre{3} \ann{1} \ann{0} - \cre{0} \cre{1} \ann{3} \ann{2}$ which needs to be considered. Instantiation of a single term is acomplished by passing a coefficient, a list representing the creation operators, and a list pertaining to the anihilation operators. Note that an operaotor containing only exciations or only de-excitations can be created by passing an empy list for unincluded type.  

> Instantiate and print the operator $\hat{\kappa}$

In [4]:
K = qf.SQOperator()
K.add_term( 1.0, [2,3], [1,0]) 
K.add_term(-1.0, [0,1], [3,2])
print(K) 

 +1.000000 ( 2^ 3^ 1 0 )
 -1.000000 ( 0^ 1^ 3 2 )



We will also need to get the MP denominator for this excitation 

> get the MP denominator for the excitation corresponding to $\hat{\kappa}_\mu$

In [5]:
# Define a list for the HF orbital energies.
orb_e = []

# Get the list of 1's and 0's representing the HF orbital occupations.
ref = H2mol.get_hf_reference()

print('\nBuilding single particle energies list:')
print('---------------------------------------')
qc = qforte.Computer(len(ref))
qc.apply_circuit(Uhf)
E0 = qc.direct_op_exp_val(H2mol.get_hamiltonian())

for i in range(len(ref)):
    qc = qforte.Computer(4)
    qc.apply_circuit(Uhf)
    qc.apply_gate(qforte.gate('X', i))
    Ei = qc.direct_op_exp_val(H2mol.get_hamiltonian())

    if(i<sum(ref)):
        ei = E0 - Ei
    else:
        ei = Ei - E0

    print(f'  {i:3}     {ei:+16.12f}')
    orb_e.append(ei)

# Define the Moller-Plesset denominator ∆_mu.   
delta_mu = np.real(orb_e[0] + orb_e[1] - orb_e[2] - orb_e[3])
print(f"\nDelta_mu:    {delta_mu:+12.10f}")


Building single particle energies list:
---------------------------------------
    0     -0.574436558130+0.000000000000j
    1     -0.574436558130+0.000000000000j
    2     +0.660910050926+0.000000000000j
    3     +0.660910050926+0.000000000000j

Delta_mu:    -2.4706932181


The PQE residuals can be expressed as the off-diagonal matrix elements of the operator $\bar{H} = \hat{U}^\dagger(\mathbf{t}) \hat{\mathcal{H}} \hat{U}(\mathbf{t})$ as $r_\mu = \bra{\Phi_\mu} \bar{H} \ket{\Phi_0}$.

Acting on the reference with the operator $e^{\theta \hat{\kappa}_\mu}$ yields the state

\begin{equation}
\ket{\Omega_\mu(\theta)} = e^{\theta \hat{\kappa}_\mu} \ket{\Phi_0} = \cos(\theta) \ket{\Phi_0} + \sin(\theta) \ket{\Phi_\mu},
\end{equation}

Taking the expectation value of the similarity transformed Hamiltonian with respect to $\Omega_\mu(\theta)$ using $\theta = \pi / 4$, and using the fact that the wave function is real, leads to the following equation for the residual elements

\begin{equation}
r_\mu = \bra{\Omega_\mu(\pi/4)} \bar{H} \ket{\Omega_\mu(\pi/4)}
- \frac{1}{2}E_\mu
- \frac{1}{2}E_0,
\end{equation}

where $E_0 = \bra{\Phi_0} \bar{H} \ket{\Phi_0}$ and $E_\mu = \bra{\Phi_\mu} \bar{H} \ket{\Phi_\mu}$.

> Define a function that returns the three quantities needed to calcualte $r_\mu$ as a function of the cluster amplitude t.

In [6]:
def get_energies(t_mu, K, Uhf):
    
    #First we need build the dUCC unitary
    Uducc, phase_pqe = trotterize(K.jw_transform(), factor=t_mu)
    
    # Initialize a QuantumComputer
    QC = qf.Computer(4)
    QC.apply_circuit(Uhf)
    QC.apply_circuit(Uducc)

    # Get the PQE energy.
    E0 = np.real(QC.direct_op_exp_val(H2mol.get_hamiltonian()))

    # Get the excited determinannt energy by applying K to the HF state.
    QC = qf.Computer(4)
    QC.apply_circuit(Uhf)
    QC.apply_operator(K.jw_transform())
    QC.apply_circuit(Uducc)
    Emu = np.real(QC.direct_op_exp_val(H2mol.get_hamiltonian()))
    
    # Re-initialize a QuantumComputer.
    QC = qf.Computer(4)
    QC.apply_circuit(Uhf)
    
    # Form the Unitary for e^{ (pi/4) Kmu }
    Uomega, phase = trotterize(K.jw_transform(), factor=np.pi/4)
    QC.apply_circuit(Uomega)

    # Get the mixed state energy
    QC.apply_circuit(Uducc)
    Eomega = np.real(QC.direct_op_exp_val(H2mol.get_hamiltonian()))

    return Eomega, Emu, E0

Now we can write a procedure that implements PQE with the quasi-Newton update!

In [7]:
# Defie the number of PQE iterations. 
pqe_iter = 11

t_mu = 0

print(f"   Iteration       Epqe                  Emu            E_omega_mu            r_mu")
print(f"--------------------------------------------------------------------------------------------")

for n in range(pqe_iter):
    
    Eomega_mu, Emu, E0 = get_energies(t_mu, K, Uhf)
    
    r_mu = Eomega_mu - 0.5*Emu - 0.5*E0
    
    print(f"       {n:2}       {E0:+12.10f}       {Emu:+12.10f}     {Eomega_mu:+12.10f}       {r_mu:+12.10f}")
    
    # Update the amplitude.
    t_mu += r_mu/delta_mu
    

    
print(f'\n\n Efci:   {H2mol.get_fci_energy():+12.10f}')

   Iteration       Epqe                  Emu            E_omega_mu            r_mu
--------------------------------------------------------------------------------------------
        0       -1.1161514489       +0.4388389026     -0.1568847366       +0.1817715366
        1       -1.1343997677       +0.4570872214     -0.2728388683       +0.0658174049
        2       -1.1367756303       +0.4594630840     -0.3153082198       +0.0233480534
        3       -1.1370743384       +0.4597617921     -0.3303959491       +0.0082603240
        4       -1.1371117228       +0.4597991765     -0.3357348320       +0.0029214411
        5       -1.1371163989       +0.4598038526     -0.3376230863       +0.0010331869
        6       -1.1371169837       +0.4598044374     -0.3382908818       +0.0003653914
        7       -1.1371170569       +0.4598045106     -0.3385270509       +0.0001292223
        8       -1.1371170660       +0.4598045197     -0.3386105731       +0.0000457000
        9       -1.1371170672   